In [1]:
from pyomo.environ import *
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash import Dash, dcc, html
import webbrowser
from threading import Timer
import dash_bootstrap_components as dbc
import plotly.io as pio
from dash.dependencies import Input, Output
import json


In [2]:
# Read price data from Excel
price_data = pd.read_excel('data/price_data_arb_reg_separate_res.xlsx', sheet_name= None, index_col=0)
schedule_data = pd.read_excel('data/schedule_data.xlsx', index_col=0)

# Parameters
Seff = 1
Ceff = 1
Deff = 1
cap_power = 1  # Example value
cap_energy = 4  # Example value
η = 1
dt = 1

λ_0 = 0.5
λ_min = 0.0
initial_soc = λ_0 * cap_energy


In [3]:
data = price_data

# Iteratively merge all dataframes on 'Timestamp'
price_df = pd.DataFrame(index =  data[list(data.keys())[0]].index)

for key in data:
    price_df = pd.concat([price_df, data[key]], axis = 1)


# Model

## Method 1 (PNNL)

In [4]:
def optimize_revenue(initial_soc, price_vector, current_period, first_period, last_period):

    # Define the model
    model = ConcreteModel()

    # Price Data for the day
    total_time_period = len(price_vector)

    T = range(1, total_time_period + 1)
    SOC_T = range(0, total_time_period + 1)

    p_arb = {t: price_vector['arb_energy_price'].values[t-1] for t in T}
    p_pres = {t: price_vector['pres_capacity_price'].values[t-1] for t in T}
    p_cres = {t: price_vector['cres_capacity_price'].values[t-1] for t in T}

    # p_reg = {t: price_vector['reg_capacity_price'].values[t-1] for t in T}
    p_reg_down = {t: price_vector['reg_down_price'].values[t-1] for t in T}
    p_reg_up = {t: price_vector['reg_up_price'].values[t-1] for t in T}


    # p_reg_e = {t: price_vector['reg_energy_price'].values[t-1] for t in T}
    # p_pres_e = {t: price_vector['pres_energy_price'].values[t-1] for t in T}
    # p_cres_e = {t: price_vector['cres_energy_price'].values[t-1] for t in T}

    start_soc = λ_0 * cap_energy

    # Variables
    model.SOC = Var(SOC_T, within=NonNegativeReals, bounds = (0, cap_energy))
 
    # model.C = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    # model.D = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.C_arb = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_arb = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.C_reg = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_reg = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_pres = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_cres = Var(T, within=NonNegativeReals, bounds = (0, 999))

    model.y_ch = Var(T, within=Binary)

    model.R_arb = Var(T, within=Reals)
    model.R_reg = Var(T, within=Reals)
    model.R_pres = Var(T, within=Reals)
    model.R_cres = Var(T, within=Reals)

    # Objective function
    model.obj = Objective(
        expr= sum(model.R_arb[t] + model.R_reg[t] + model.R_pres[t] + model.R_cres[t] for t in T),        
        sense=maximize
    )

    def revenue_arbitrage_rule(model, t):
        return model.R_arb[t] == p_arb[t]  * (model.D_arb[t] - model.C_arb[t]) * dt
    model.revenue_arbitrage = Constraint(T, rule=revenue_arbitrage_rule)

    def revenue_regulation_rule(model, t):
        #return model.R_reg[t] == 0.9 * (p_reg[t] * (model.C_reg[t] + model.D_reg[t])) * dt
        #return model.R_reg[t] == 0.95 * (p_reg[t] * (model.D_reg[t])) * dt
        return model.R_reg[t] == 0.95 * (p_reg_down[t] * model.C_reg[t] + p_reg_up[t] * model.D_reg[t]) * dt

    model.revenue_regulation = Constraint(T, rule=revenue_regulation_rule)

    def revenue_primary_reserve_rule(model, t):
        return model.R_pres[t] == p_pres[t]  * model.D_pres[t] * dt
    model.revenue_preserve = Constraint(T, rule=revenue_primary_reserve_rule)

    def revenue_contingency_reserve_rule(model, t):
        return model.R_cres[t] == p_cres[t]  * model.D_cres[t] * dt
    model.revenue_creserve = Constraint(T, rule=revenue_contingency_reserve_rule)

    # Constraints
    def soc_constraints(model, t):
        if t == 0:
            return model.SOC[t] == initial_soc
        else:
            # return model.SOC[t] == model.SOC[t-1] * Seff + ((model.C_arb[t] + 0.15 * model.C_reg[t]) * Ceff - (model.D_arb[t] + 0.15 * model.D_reg[t])/Deff) * dt
            return model.SOC[t] == model.SOC[t-1] * Seff + (model.C_arb[t]  * Ceff - model.D_arb[t]/Deff) * dt
    model.SOC_constraints = Constraint(SOC_T, rule=soc_constraints)

    if current_period == last_period:

        def soc_final_state_constraint(model):
            end_t = SOC_T[-1]
            return model.SOC[end_t] == start_soc #* Seff 
        model.SOC_final = Constraint(rule=soc_final_state_constraint)

    def total_charge_rule(model, t):
        return model.C_reg[t] - (model.D_arb[t] - model.C_arb[t])  <= cap_power 
        # return model.C_reg[t] + model.C_arb[t]  <= cap_power 
    model.total_charge = Constraint(T, rule=total_charge_rule)

    def total_discharge_rule(model, t):
        # return model.D_arb[t] + model.D_reg[t] + model.D_pres[t] + model.D_cres[t] <= cap_power * (1 - model.y_ch[t])
        return model.D_reg[t] + (model.D_arb[t] - model.C_arb[t]) +  model.D_pres[t] +  model.D_cres[t] <= cap_power 
        # return model.D_reg[t] + model.D_arb[t]  <= cap_power 
    model.total_discharge = Constraint(T, rule=total_discharge_rule)

    def soc_cap_constraint(model, t):
        return model.SOC[t] >= cap_energy * λ_min
    model.SOC_cap = Constraint(SOC_T, rule=soc_cap_constraint)

    def energy_cap_C_constraint(model):
        return sum(model.C_arb[t] +  model.C_reg[t]*0.15 for t in T)*dt <= η * cap_energy
        # return sum(model.C_arb[t] for t in T)*dt <= η * cap_energy
    model.energy_cap_C = Constraint(rule=energy_cap_C_constraint)

    def energy_cap_D_constraint(model):
        return sum(model.D_arb[t] + model.D_reg[t]*0.15 + model.D_cres[t] for t in T)*dt <= η * cap_energy
        # return sum(model.D_arb[t] for t in T)*dt <= η * cap_energy
    model.energy_cap_D = Constraint(rule=energy_cap_D_constraint)

    def c_limit_constraint(model, t):
        return model.C_arb[t] <= cap_power * model.y_ch[t]
    model.C_limit = Constraint(T, rule=c_limit_constraint)

    def d_limit_constraint(model, t):
        return model.D_arb[t]  <= cap_power * (1 - model.y_ch[t])
    model.D_limit = Constraint(T, rule=d_limit_constraint)

    # def arb_schedule_constraint_rule(model, t):
    #     return model.C_arb[t] + model.D_arb[t] <= schedule_data.loc[t, 'arb'] * cap_power
    # model.arb_schedule = Constraint(T, rule=arb_schedule_constraint_rule)

    # def reg_constraint_rule(model, t):
    #     return model.C_reg[t] == model.D_reg[t]
    # model.reg_up_down = Constraint(T, rule=reg_constraint_rule)

    def soc_reg_down_rule(model, t):
        return model.SOC[t] + 0.15 * model.C_reg[t] * Ceff * dt <= cap_energy
    model.soc_reg_down = Constraint(T, rule=soc_reg_down_rule)

    def soc_reg_up_rule(model, t):
        return model.SOC[t] >= (0.15 * model.D_reg[t] + model.D_pres[t] + model.D_cres[t])/Deff * dt
    model.soc_reg_up = Constraint(T, rule=soc_reg_up_rule)


    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model, tee=True)

    # Extract results for charging, discharging, and SOC
    charging_schedule = [model.C_arb[t].value for t in T]  # Charging as negative
    discharging_schedule = [model.D_arb[t].value for t in T]  # Discharging as positive
    reg_down_schedule = [model.C_reg[t].value for t in T]  # Charging as negative
    reg_up_schedule = [model.D_reg[t].value for t in T]  # Discharging as positive
    pres_schedule = [model.D_pres[t].value for t in T]  # Charging as negative
    cres_schedule = [model.D_cres[t].value for t in T]  # Discharging as positive

    soc_schedule = [model.SOC[t].value for t in SOC_T]  # State of Charge

    rev_arbitrage =  sum(model.R_arb[t] for t in T)() 
    rev_regulation = sum(model.R_reg[t] for t in T)() 
    rev_cont_reserve = sum(model.R_cres[t] for t in T)() 

    # Return the results and final SOC
    return model.obj(), [charging_schedule, discharging_schedule, soc_schedule, reg_down_schedule, reg_up_schedule, pres_schedule, cres_schedule], [rev_arbitrage, rev_regulation, rev_cont_reserve]

## Method 2 (Main)

In [5]:
def optimize_revenue2(initial_soc, price_vector, current_period, first_period, last_period):

    # Define the model
    model = ConcreteModel()

    # Price Data for the day
    total_time_period = len(price_vector)

    T = range(1, total_time_period + 1)
    SOC_T = range(0, total_time_period + 1)

    p_arb = {t: price_vector['arb_energy_price'].values[t-1] for t in T}
    p_pres = {t: price_vector['pres_capacity_price'].values[t-1] for t in T}
    p_cres = {t: price_vector['cres_capacity_price'].values[t-1] for t in T}

    # p_reg = {t: price_vector['reg_capacity_price'].values[t-1] for t in T}
    p_reg_down = {t: price_vector['reg_down_price'].values[t-1] for t in T}
    p_reg_up = {t: price_vector['reg_up_price'].values[t-1] for t in T}


    # p_reg_e = {t: price_vector['reg_energy_price'].values[t-1] for t in T}
    # p_pres_e = {t: price_vector['pres_energy_price'].values[t-1] for t in T}
    # p_cres_e = {t: price_vector['cres_energy_price'].values[t-1] for t in T}

    start_soc = λ_0 * cap_energy

    # Variables
    model.SOC = Var(SOC_T, within=NonNegativeReals, bounds = (0, cap_energy))
 
    # model.C = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    # model.D = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.C_arb = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_arb = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.C_reg = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_reg = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_pres = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_cres = Var(T, within=NonNegativeReals, bounds = (0, 999))

    model.y_ch = Var(T, within=Binary)

    model.R_arb = Var(T, within=Reals)
    model.R_reg = Var(T, within=Reals)
    model.R_pres = Var(T, within=Reals)
    model.R_cres = Var(T, within=Reals)

    # Objective function
    model.obj = Objective(
        expr= sum(model.R_arb[t] + model.R_reg[t] + model.R_pres[t] + model.R_cres[t] for t in T),        
        sense=maximize
    )

    def revenue_arbitrage_rule(model, t):
        return model.R_arb[t] == p_arb[t]  * (model.D_arb[t] - model.C_arb[t]) * dt
    model.revenue_arbitrage = Constraint(T, rule=revenue_arbitrage_rule)

    def revenue_regulation_rule(model, t):
        #return model.R_reg[t] == 0.9 * (p_reg[t] * (model.C_reg[t] + model.D_reg[t])) * dt
        #return model.R_reg[t] == 0.95 * (p_reg[t] * (model.D_reg[t])) * dt
        return model.R_reg[t] == 0.95 * (p_reg_down[t] * model.C_reg[t] + p_reg_up[t] * model.D_reg[t]) * dt

    model.revenue_regulation = Constraint(T, rule=revenue_regulation_rule)

    def revenue_primary_reserve_rule(model, t):
        return model.R_pres[t] == p_pres[t]  * model.D_pres[t] * dt
    model.revenue_preserve = Constraint(T, rule=revenue_primary_reserve_rule)

    def revenue_contingency_reserve_rule(model, t):
        return model.R_cres[t] == p_cres[t]  * model.D_cres[t] * dt
    model.revenue_creserve = Constraint(T, rule=revenue_contingency_reserve_rule)

    # Constraints
    def soc_constraints(model, t):
        if t == 0:
            return model.SOC[t] == initial_soc
        else:
            # return model.SOC[t] == model.SOC[t-1] * Seff + ((model.C_arb[t] + 0.15 * model.C_reg[t]) * Ceff - (model.D_arb[t] + 0.15 * model.D_reg[t])/Deff) * dt
            return model.SOC[t] == model.SOC[t-1] * Seff + (model.C_arb[t]  * Ceff - model.D_arb[t]/Deff) * dt
    model.SOC_constraints = Constraint(SOC_T, rule=soc_constraints)

    if current_period == last_period:

        def soc_final_state_constraint(model):
            end_t = SOC_T[-1]
            return model.SOC[end_t] == start_soc #* Seff 
        model.SOC_final = Constraint(rule=soc_final_state_constraint)

    def total_charge_rule(model, t):
        return model.C_reg[t] + model.C_arb[t]  <= cap_power * model.y_ch[t]
        # return model.C_reg[t] + model.C_arb[t]  <= cap_power 
    model.total_charge = Constraint(T, rule=total_charge_rule)

    def total_discharge_rule(model, t):
        # return model.D_arb[t] + model.D_reg[t] + model.D_pres[t] + model.D_cres[t] <= cap_power * (1 - model.y_ch[t])
        return model.D_reg[t] + model.D_arb[t] +  model.D_pres[t] +  model.D_cres[t] <= cap_power * (1 - model.y_ch[t])
        # return model.D_reg[t] + model.D_arb[t]  <= cap_power 
    model.total_discharge = Constraint(T, rule=total_discharge_rule)

    def soc_cap_constraint(model, t):
        return model.SOC[t] >= cap_energy * λ_min
    model.SOC_cap = Constraint(SOC_T, rule=soc_cap_constraint)

    def energy_cap_C_constraint(model):
        return sum(model.C_arb[t] +  model.C_reg[t]*0.15 for t in T)*dt <= η * cap_energy
        # return sum(model.C_arb[t] for t in T)*dt <= η * cap_energy
    model.energy_cap_C = Constraint(rule=energy_cap_C_constraint)

    def energy_cap_D_constraint(model):
        return sum(model.D_arb[t] + model.D_reg[t]*0.15 for t in T)*dt <= η * cap_energy
        # return sum(model.D_arb[t] for t in T)*dt <= η * cap_energy
    model.energy_cap_D = Constraint(rule=energy_cap_D_constraint)

    def c_limit_constraint(model, t):
        return model.C_arb[t] <= cap_power * model.y_ch[t]
    model.C_limit = Constraint(T, rule=c_limit_constraint)

    def d_limit_constraint(model, t):
        return model.D_arb[t]  <= cap_power * (1 - model.y_ch[t])
    model.D_limit = Constraint(T, rule=d_limit_constraint)

    # def arb_schedule_constraint_rule(model, t):
    #     return model.C_arb[t] + model.D_arb[t] <= schedule_data.loc[t, 'arb'] * cap_power
    # model.arb_schedule = Constraint(T, rule=arb_schedule_constraint_rule)

    # def reg_constraint_rule(model, t):
    #     return model.C_reg[t] == model.D_reg[t]
    # model.reg_up_down = Constraint(T, rule=reg_constraint_rule)

    def soc_reg_down_rule(model, t):
        return model.SOC[t] + 0.15 * model.C_reg[t] * Ceff * dt <= cap_energy
    model.soc_reg_down = Constraint(T, rule=soc_reg_down_rule)

    def soc_reg_up_rule(model, t):
        return model.SOC[t] >= (0.15 * model.D_reg[t] + model.D_pres[t] + model.D_cres[t])/Deff * dt
    model.soc_reg_up = Constraint(T, rule=soc_reg_up_rule)


    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model, tee=True)

    # Extract results for charging, discharging, and SOC
    charging_schedule = [model.C_arb[t].value for t in T]  # Charging as negative
    discharging_schedule = [model.D_arb[t].value for t in T]  # Discharging as positive
    reg_down_schedule = [model.C_reg[t].value for t in T]  # Charging as negative
    reg_up_schedule = [model.D_reg[t].value for t in T]  # Discharging as positive
    pres_schedule = [model.D_pres[t].value for t in T]  # Charging as negative
    cres_schedule = [model.D_cres[t].value for t in T]  # Discharging as positive

    soc_schedule = [model.SOC[t].value for t in SOC_T]  # State of Charge

    rev_arbitrage =  sum(model.R_arb[t] for t in T)() 
    rev_regulation = sum(model.R_reg[t] for t in T)() 
    rev_cont_reserve = sum(model.R_cres[t] for t in T)() 

    # Return the results and final SOC
    return model.obj(), [charging_schedule, discharging_schedule, soc_schedule, reg_down_schedule, reg_up_schedule, pres_schedule, cres_schedule], [rev_arbitrage, rev_regulation, rev_cont_reserve]

# Run

In [6]:
time_period = 24    # periods

total_time_period = len(price_df)

num_slices = total_time_period // time_period

# Lists to store results
total_revenue = 0
arbitrage = 0
regulation = 0
creserve = 0

all_charging_schedules = []
all_discharging_schedules = []
all_reg_down_schedules = []
all_reg_up_schedules = []
all_pres_schedules = []
all_cres_schedules = []
all_soc_schedules = []

first_p = 0
last_p = num_slices - 1

# Run the optimization for each time period
for p in range(num_slices):
    print("watch", p)
    periodic_price = price_df[p*time_period:(p+1)*time_period]
    revenue, [charging_schedule, discharging_schedule, soc_schedule, reg_down_schedule, reg_up_schedule, pres_schedule, cres_schedule], [rev_arbitrage, rev_regulation, rev_cont_reserve] = optimize_revenue2(initial_soc, periodic_price, p, first_p, last_p)
    final_soc = soc_schedule[-1]
    
    # Store the results
    total_revenue += revenue
    arbitrage += rev_arbitrage
    regulation += rev_regulation
    creserve += rev_cont_reserve

    all_charging_schedules.extend(charging_schedule)
    all_discharging_schedules.extend(discharging_schedule)
    all_reg_down_schedules.extend(reg_down_schedule)
    all_reg_up_schedules.extend(reg_up_schedule)
    all_pres_schedules.extend(pres_schedule)
    all_cres_schedules.extend(cres_schedule)
    all_soc_schedules.extend(soc_schedule[:-1])

    # Update the initial SOC for the next day
    initial_soc = final_soc

print(total_revenue)

watch 0
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmpdpqhovv6.glpk.raw
 --wglp /var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmpj8oyej85.glpk.glp
 --cpxlp /var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmp2kbk8u25.pyomo.lp
Reading problem data from '/var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmp2kbk8u25.pyomo.lp'...
/var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmp2kbk8u25.pyomo.lp:2113: warning: lower bound of variable 'x341' redefined
/var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmp2kbk8u25.pyomo.lp:2113: warning: upper bound of variable 'x341' redefined
292 rows, 289 columns, 842 non-zeros
24 integer variables, all of which are binary
2137 lines were read
Writing problem data to '/var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmpj8oyej85.glpk.glp'...
1959 lines were written
GLPK Integer Optimizer 5.0
292 rows, 289 columns, 842 non-zeros
24 integer variables, all of which ar

In [7]:
arbitrage

40733.03990869175

In [8]:
regulation

107429.83579357581

In [9]:
creserve

16679.52137362959

In [10]:
# Create a DataFrame for the results
data = {
    't': range(1, len(all_charging_schedules) + 1),
    'charge': all_charging_schedules,
    'discharge': all_discharging_schedules,
    'reg_down': all_reg_down_schedules,
    'reg_up': all_reg_up_schedules,
    'pres': all_pres_schedules,
    'cres': all_cres_schedules,
    'soc': all_soc_schedules
}

result_df = pd.DataFrame(data)

result_df = result_df.set_index(['t'])

result_df['net_discharge'] = result_df['discharge'] - result_df['charge']
result_df['soc_percent'] = result_df['soc'] /cap_energy

result_df

,charge,discharge,reg_down,reg_up,pres,cres,soc,net_discharge,soc_percent
t,,,,,,,,,
1,0.0,0.0,-2.220446e-16,1.0,0.0,0.0,2.0,0.0,0.50
2,1.0,0.0,0.000000e+00,0.0,0.0,0.0,2.0,-1.0,0.50
3,0.0,0.0,1.000000e+00,0.0,0.0,0.0,3.0,0.0,0.75
4,0.0,0.0,1.000000e+00,0.0,0.0,0.0,3.0,0.0,0.75
5,0.0,0.0,1.000000e+00,0.0,0.0,0.0,3.0,0.0,0.75
...,...,...,...,...,...,...,...,...,...
8756,0.0,0.0,0.000000e+00,1.0,0.0,0.0,1.0,0.0,0.25
8757,1.0,0.0,0.000000e+00,0.0,0.0,0.0,1.0,-1.0,0.25
8758,0.0,0.0,0.000000e+00,1.0,0.0,0.0,2.0,0.0,0.50


In [11]:
result_df.reg_up.max()

1.00000000000001

In [12]:
result_df[result_df.reg_up < 0.9]

,charge,discharge,reg_down,reg_up,pres,cres,soc,net_discharge,soc_percent
t,,,,,,,,,
2,1.0,0.0,0.0,0.0,0.0,0.0,2.0,-1.0,0.50
3,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.75
4,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.75
5,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.75
6,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.75
...,...,...,...,...,...,...,...,...,...
8748,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.25
8750,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.25
8751,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.25


In [13]:
# Create interactive plot for charging and discharging
fig_charge_discharge = go.Figure()
fig_charge_discharge.add_trace(go.Scatter(x=result_df.index, y=result_df['net_discharge'], mode='lines', name='net'))
fig_charge_discharge.update_layout(
    title='Charging and Discharging Schedule',
    xaxis_title='Time',
    yaxis_title='Power (MW)',
    # xaxis=dict(
    #     tickmode='linear',
    #     dtick=86400000.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    # ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)
fig_charge_discharge.show()

# Create interactive plot for charging and discharging
fig_soc = go.Figure()
fig_soc.add_trace(go.Scatter(x=result_df.index, y=result_df['soc_percent']*100, mode='lines', name='net', line=dict(color='orange')))
fig_soc.update_layout(
    title='State of Charge',
    xaxis_title='Time',
    yaxis_title='SOC (%)',
    # xaxis=dict(
    #     tickmode='linear',
    #     dtick=86400000.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    # ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)
fig_soc.show()

In [14]:
# Create the combined layout
combined_html = f"""
<html>
    <head>
        <title>BESS Revenue Simulation Dashboard</title>
        <style>
            .responsive-plot {{
                width: 80%;
                max-width: 3600px;
                height: 80%;
                max-height: 600px;
                margin: auto;
            }}
        </style>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>

    </head>
    <body>
        <h1>BESS Revenue Simulation Dashboard</h1>
        <div>
            <h3>Charging and Discharging Schedules</h3>
            <div id="plot1" class="responsive-plot">
                {pio.to_html(fig_charge_discharge, include_plotlyjs=False, full_html=False)}
            </div>
        </div>
        <div>
            <h3>State of Charge Over Time</h3>
            <div id="plot2" class="responsive-plot">
                {pio.to_html(fig_soc, include_plotlyjs=False, full_html=False)}
            </div>
        </div>
    </body>
</html>
"""

# Save the combined HTML to a file
with open("bess_dashboard.html", "w") as f:
    f.write(combined_html)